In [30]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os, random

from tensorflow.keras.layers import Dense, LSTM, Input, TimeDistributed, GRU, SimpleRNN, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Bidirectional, Activation

In [2]:
# bd sn ht mt lt oh ch cr ri
checking_arr = [
    [0, 0, 0, 0, 0, 0, 0, 0, 0], # 0 null
    [1, 0, 0, 0, 0, 0, 0, 0, 0], # 1  B
    [1, 0, 0, 0, 0, 0, 0, 1, 0], # 2  B+C
    [1, 0, 0, 0, 0, 0, 1, 0, 0], # 3  B+CH
    [1, 0, 0, 0, 1, 0, 0, 0, 0], # 5  B+FT
    [1, 0, 1, 0, 0, 0, 0, 0, 0], # 4  B+HT
    [1, 0, 0, 1, 0, 0, 0, 0, 0], # 6  B+MT
    [1, 0, 0, 0, 0, 1, 0, 0, 0], # 7  B+OH
    [1, 0, 0, 0, 0, 0, 0, 0, 1], # 8  B+R
    [0, 0, 0, 0, 0, 0, 1, 0, 0], # 9  CH
    [0, 0, 0, 0, 1, 0, 1, 0, 0], # 11 CH+FT
    [0, 0, 1, 0, 0, 0, 1, 0, 0], # 10 CH+HT
    [0, 0, 0, 0, 0, 0, 1, 0, 1], # 12 CH+R
    [0, 0, 0, 0, 1, 0, 0, 0, 0], # 15 FT
    [0, 0, 1, 0, 0, 0, 0, 0, 0], # 13 HT
    [0, 0, 1, 0, 1, 0, 0, 0, 0], # 14 HT+FT
    [0, 0, 0, 1, 0, 0, 0, 0, 0], # 16 MT
    [0, 0, 0, 0, 0, 1, 0, 0, 0], # 17 OH
    [0, 0, 0, 0, 0, 0, 0, 0, 1], # 18 R
    [0, 1, 0, 0, 0, 0, 0, 0, 0], # 19 S
    [1, 1, 0, 0, 0, 0, 0, 0, 0], # 20 S+B
    [1, 1, 0, 0, 0, 0, 1, 0, 0], # 21 S+B+CH
    [1, 1, 0, 0, 1, 0, 0, 0, 0], # 22 S+B+FT
    [1, 1, 0, 0, 0, 1, 0, 0, 0], # 23 S+B+OH
    [1, 1, 0, 0, 0, 0, 0, 0, 1], # 24 S+B+R
    [0, 1, 0, 0, 0, 0, 0, 1, 0], # 25 S+C
    [0, 1, 0, 0, 0, 0, 1, 0, 0], # 26 S+CH
    [0, 1, 0, 0, 1, 0, 0, 0, 0], # 27 S+FT
    [0, 1, 0, 0, 0, 1, 0, 0, 0], # 28 S+OH
    [0, 1, 0, 0, 0, 0, 0, 0, 1], # 29 S+R
    # [0, 0, 0, 0, 0, 0, 0, 1, 0], # 30 C
]


In [114]:
train_x = []
train_y = []

barss = os.listdir('./bar2singlelabel_0111_/')
random.shuffle(barss)
for i in barss:
    tmpp = pd.read_csv('./bar2singlelabel_0111_/' + i, compression='gzip').to_numpy()
    #### if tmp[-1][0] != 1:
    tmp = []
    for j in tmpp:
        if 1 in j:
            tmp.append(checking_arr[list(j).index(1)+1])
        else:
            tmp.append(checking_arr[0])
    train_x.append(tmp[:-1])
    train_y.append(tmp)

train_x = np.array(train_x)
train_y = np.array(train_y)



In [121]:
encoder_inputs = Input(shape=(None,29))
encoder_lstm = Bidirectional(LSTM(units=64, return_state=True, dropout=0.2, recurrent_dropout=0.2))
# bid1 = LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
# bid2 = LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, go_backwards=True)
# b = Bidirectional(bid1, backward_layer=bid2, input_shape=(32,29))

encoder_outputs, f_state_h, f_state_c, b_state_h, b_state_c = encoder_lstm(encoder_inputs)

state_h = Concatenate()([f_state_h, b_state_h])
state_c = Concatenate()([f_state_c, b_state_c])

encoder_states = [state_h, state_c]

In [122]:
decoder_inputs = Input(shape=(None,29))
decoder_lstm = LSTM(units=128,return_sequences=True, return_state=True, dropout=0.2, recurrent_dropout=0.2)

# dbid1 = LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2)
# dbid2 = LSTM(units=128, return_sequences=True, dropout=0.2, recurrent_dropout=0.2, go_backwards=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state = encoder_states)

decoder_dense = Dense(29, activation='sigmoid')

decoder_outputs = decoder_dense(decoder_outputs)


In [123]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)


# # define inference encoder
# encoder_model = Model(encoder_inputs, encoder_states)
# # define inference decoder
# decoder_state_input_h = Input(shape=(64*2,))
# decoder_state_input_c = Input(shape=(64*2,))
# decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
# decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# decoder_states = [state_h, state_c]
# decoder_outputs = decoder_dense(decoder_outputs)
# decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [124]:
model.load_weights('./chkpnt/0210/seq2seq_single/cp-0025.ckpt')

In [118]:
chkpnt_pth = './chkpnt/0210/seq2seq_multi/cp-{epoch:04d}.ckpt'
chkpnt_dir = os.path.dirname(chkpnt_pth)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=chkpnt_pth, save_weights_only=True, verbose=1)

In [119]:
model.compile(loss = tf.keras.losses.MSE, optimizer='adam', metrics=['accuracy'])
callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7)

In [120]:
with tf.device("/gpu:0"):
    model.fit([train_x,train_y],train_y, epochs=4000, callbacks=[callback, cp_callback],validation_split=0.1)

Epoch 1/4000
275/275 [==============================] - 150s 534ms/step - loss: 0.0664 - accuracy: 0.4041 - val_loss: 0.0309 - val_accuracy: 0.5043

Epoch 00001: saving model to ./chkpnt/0210/seq2seq_multi\cp-0001.ckpt
Epoch 2/4000
275/275 [==============================] - 142s 518ms/step - loss: 0.0218 - accuracy: 0.5616 - val_loss: 0.0083 - val_accuracy: 0.5676

Epoch 00002: saving model to ./chkpnt/0210/seq2seq_multi\cp-0002.ckpt
Epoch 3/4000
275/275 [==============================] - 143s 521ms/step - loss: 0.0104 - accuracy: 0.5953 - val_loss: 0.0034 - val_accuracy: 0.5964

Epoch 00003: saving model to ./chkpnt/0210/seq2seq_multi\cp-0003.ckpt
Epoch 4/4000
275/275 [==============================] - 142s 518ms/step - loss: 0.0064 - accuracy: 0.6080 - val_loss: 0.0021 - val_accuracy: 0.5951

Epoch 00004: saving model to ./chkpnt/0210/seq2seq_multi\cp-0004.ckpt
Epoch 5/4000
275/275 [==============================] - 148s 539ms/step - loss: 0.0048 - accuracy: 0.6182 - val_loss: 0.0017

In [ ]:
01 : 25Epoch
02 : 61Epoch

In [ ]:
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [127]:
decoder_state_input_h = Input(shape=(128,))
decoder_state_input_c = Input(shape=(128,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용.
# 뒤의 함수 decode_sequence()에 동작을 구현 예정
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)

# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태를 버리지 않음.
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [ ]:
def decode_sequence(input_seq):
      # 입력으로부터 인코더의 상태를 얻음
  states_value = encoder_model.predict(input_seq)

  # <SOS>에 해당하는 원-핫 벡터 생성
  # target_seq = np.zeros((1, 1, tar_vocab_size))
  # target_seq[0, 0, tar_to_index['\t']] = 1.
  target_seq = input_seq[0]

  stop_condition = False
  decoded_sentence = ""

  # stop_condition이 True가 될 때까지 루프 반복
  # while not stop_condition:
  while _ in range(32):
    # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
    output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

    # 예측 결과를 문자로 변환
    sampled_token_index = np.argmax(output_tokens[0, -1, :])
    sampled_char = index_to_tar[sampled_token_index]

    # 현재 시점의 예측 문자를 예측 문장에 추가
    decoded_sentence += sampled_char
    
    # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, sampled_token_index] = 1.

    # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
    states_value = [h, c]

  return decoded_sentence